# am

In [ ]:
// // test

open testing

()



## sum

In [ ]:
inl sum (a' : a _ _) =
    a' |> am.fold (+) 0

()



In [ ]:
// // test

am.init 10i32 id
|> sum
|> _equal 45

type Mut0 = {mutable l0 : int32}
and Mut1 = {mutable l0 : int32; mutable l1 : int32}
let rec method1 (v0 : Mut0) : bool =
    let v1 : int32 = v0.l0
    let v2 : bool = v1 < 10
    v2
and method2 (v0 : int32, v1 : Mut1) : bool =
    let v2 : int32 = v1.l0
    let v3 : bool = v2 < v0
    v3
and method0 () : unit =
    let v0 : (int32 []) = Array.zeroCreate<int32> (10)
    let v1 : Mut0 = {l0 = 0} : Mut0
    while method1(v1) do
        let v3 : int32 = v1.l0
        v0.[int v3] <- v3
        let v4 : int32 = v3 + 1
        v1.l0 <- v4
        ()
    let v5 : int32 = v0.Length
    let v6 : Mut1 = {l0 = 0; l1 = 0} : Mut1
    while method2(v5, v6) do
        let v8 : int32 = v6.l0
        let v9 : int32 = v6.l1
        let v10 : int32 = v0.[int v8]
        let v11 : int32 = v9 + v10
        let v12 : int32 = v8 + 1
        v6.l0 <- v12
        v6.l1 <- v11
        ()
    let v13 : int32 = v6.l1
    let v14 : bool = v13 = 45
    let v15 : string = $"_equal / actual: %A{v13} / expected: %A{4

## init_series

In [ ]:
inl init_series start end inc =
    inl total = conv ((end - start) / inc) + 1
    am.init total (conv >> (*) inc >> (+) start) : a i32 _

()



In [ ]:
// // test

init_series 0 1 0.5
|> _equal (a ;[0f64; 0.5; 1])

type Mut0 = {mutable l0 : int32}
let rec method1 (v0 : Mut0) : bool =
    let v1 : int32 = v0.l0
    let v2 : bool = v1 < 3
    v2
and method2 (v0 : (float []), v1 : (float []), v2 : int32) : bool =
    let v3 : int32 = v0.Length
    let v4 : bool = v2 < v3
    if v4 then
        let v5 : float = v0.[int v2]
        let v6 : float = v1.[int v2]
        let v7 : bool = v5 = v6
        if v7 then
            let v8 : int32 = v2 + 1
            method2(v0, v1, v8)
        else
            false
    else
        true
and method0 () : unit =
    let v0 : (float []) = Array.zeroCreate<float> (3)
    let v1 : Mut0 = {l0 = 0} : Mut0
    while method1(v1) do
        let v3 : int32 = v1.l0
        let v4 : float = float v3
        let v5 : float = 0.5 * v4
        v0.[int v3] <- v5
        let v6 : int32 = v3 + 1
        v1.l0 <- v6
        ()
    let v7 : (float []) = [|0.0; 0.5; 1.0|]
    let v8 : int32 = v0.Length
    let v9 : int32 = v7.Length
    let v10 : bool = v8 = v9
    let v11 : bool 